# Training and Vizualizing a Decision Tree

To understand Decision Trees we start by building one.

In [9]:
# Training the decision Tree
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:,2:] # We get the petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth = 2)
tree_clf.fit(X,y)

DecisionTreeClassifier(max_depth=2)

In [23]:
# Exporting a graph
from sklearn.tree import export_graphviz

f = open("/home/omar/Bureau/apprentissage/Géron/Chapter_6_ressources/iris_tree.dot", 'w')
export_graphviz(
    tree_clf,
    out_file = f,
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded = True,
    filled = True
)

# Command to convert to a png : dot -Tpng iris_tree.dot -o iris_tree.png
# Command to open it : eog <img>.png

# Making predictions

![Decision Tree](Chapter_6_ressources/iris_tree.png)

We start at the root node. If the petal length is <= 2.45 we go left. The left node happens to be a leaf node, meaning without any children, so it simply predicts the class it has, the $setosa$. On the contrary, if it has a petal length greater than 2.45 it goes right, then checks the condition about the width and the predicts.

A nodes samples attribute is the number of nodes it applies to. The Values attribute gives how many instances of each class the node applies to. The Gini coefficient gives the purity/impurity of the node. A node is pure if all the instances it applies to are in the same class. The gini coefficient is 1 - the ratios of the instances in class k over the samples of node i. 

The Class Probabilities are given by this ratio. It is the same anywhere in the node. 

## The CART training algorithm

The Cart training algorithm starts by splitting the data set according to a single feature k. It also assigns a threshold to this feature. The class and threshold is searched in order to produce the purest subsets, and the cost function is minimized iteratively over the nodes. 

The CART algorithm is a greedy algorithm: it searches for an optimum split at the top level then repeats the process at each level. 

## Computational Complexity

## Gini Impurity or Entropy

The Gini Impurity is used by default but we can choose to use entropy. Both produce similar results, while Gini is faster to compute and tends to isolate the most frequent class in its own branch while the entropy produces more balanced trees. 

## Regularization Hyperparameters 

Decision Trees make very few assumptions about the training unlike for example the linear regression where the data is assumed to be linear.

In the decision tree, a non parametric model, the number of parameters is not predetermined so its easy to overfit the data. To avoid doing so, it is possible to restrict the models parameters, by for instance reducing maximum depth of the model.

# Regression 

The regression tree is very similar to the classification tree. The value predicted is not the most frequent class, it is rather the average value of the instances in the node. 

The training is the same, except it aims to minimize the mse. It is important to set the minimum number of samples per leafs to avoid overfitting, which will happen without regularization. 

# Instability

Decision Trees love orthogonal decision boundaries which makes them sensitive to training set rotation. They are also very sensitive to data variation, leading to a big variance. 

# Exercices

In [198]:
# Training and fine tuning a decision tree on the moons data set

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

moons = make_moons(n_samples = 10000, noise = 0.4)
train_X, test_X, train_y, test_y = train_test_split(moons[0],moons[1], test_size = 0.2)


tree_clf = DecisionTreeClassifier()

param_search = [{'max_leaf_nodes': [3,5,7,9,11,13,15]},]

grid_search = GridSearchCV(tree_clf, param_search, cv = 5, scoring = 'accuracy')

grid_search.fit(train_X, train_y)

tree_clf = grid_search.best_estimator_
params = grid_search.best_params_
print("Best Parameters :", params)

prediction = tree_clf.predict(test_X)

from sklearn.metrics import f1_score

print("Accuracy :",f1_score(prediction, test_y))

Best Parameters : {'max_leaf_nodes': 15}
Accuracy : 0.8511875908870576


In [199]:
# Growing a forest 

from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(train_X) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(train_X):
    X_mini_train = train_X[mini_train_index]
    y_mini_train = train_y[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [200]:
precisions = []

from sklearn.base import clone
tree = clone(grid_search.best_estimator_)
for sets in mini_sets:
    tree.fit(sets[0],sets[1])
    pred = tree.predict(test_X)
    precisions.append(f1_score(pred, test_y))

import numpy as np
print("Precision :", np.mean(precisions))

Precision : 0.7952453531994995


In [201]:
predictions = []

from sklearn.base import clone

for sets in mini_sets:
    tree = clone(grid_search.best_estimator_)
    tree.fit(sets[0],sets[1])
    pred = tree.predict(test_X)
    predictions.append(pred)

from scipy.stats import mode

common = mode(predictions, axis=0)
common = common[0].tolist()
print("Accuracy :", f1_score(common[0], test_y))

Accuracy : 0.8476953907815632
